# Use ButcherPy's NMF algorithm

This vignette shows the basic workflow to use NMF from the ButcherPy package. The package is compatible to various datatypes, which are all presented here. We specifically show an example with the RData files on bipolar disorder data provided in the GitHub repository of ButcherPy.

In [ ]:
import numpy as np
import anndata as ad
import scanpy as sc
import pandas as pd
import src.butcherPy.multiplerun_NMF_class as multinmf
import src.butcherPy.nmf_run as nmf
from src.modules.utils import rds_to_ann

## Load Data

You can use a simple numpy array, but note that you cannot provide annotations in this way. Default annotations in the form of "Gene_x" are used for the rows and "Sample_x" are used for the columns.

In [ ]:
np.random.seed(123)
# matrix with 1000 rows corresponding to 1000 genes and 6 columns corresponding to samples
test_mat = np.random.rand(1000,6)

You can use a pandas Dataframe. Make sure that the genes are stored in the rows and samples in the columns. If it is the other way around use the transpose function.

In [ ]:
df = pd.DataFrame(test_mat)
# df = df.tranpose()
df.index = ["Gene_" + str(i+1) for i in range(df.shape[0])]
df.columns = ["Sample_" + str(i+1) for i in range(df.shape[1])]

You can use an AnnData object with gene annotations in the "var_names" variable and sample annotations in the "obs_names" variable.

In [ ]:
path_to_adata = "your_path.h5ad"
adata = sc.read_h5ad(path_to_adata)

You can use RData that will be transformed to AnnData for you. Here the package provides a few options:
1. Only provide a path to RData file that stores the matrix. In this case the default annoations are created the same way, as if using a numpy array. Be aware, that the matrix should have the form (samples, genes), meaning that the genes are stored in the columns. Thus, make sure to provide a boolean value for gene_columns; True if the genes are stored in the columns and False if not.
2. Provide paths to RData files containing the matrix, the gene and sample annotations. You have to provide the gene_columns variable here as well. But if you are unsure about the stored format, this option provides you with warnings, in case the matrix and annotation dimensions do not coincide.
3. Provide path to RData file containing the matrix, and self extracted annotations. This is the best option, if the function fails with the provided data files to the gene and sample annotations. If you are unsure how to extract your annotations from RData files, you can have a look in the next cell of code providing a small tutorial how to use the rds2py package to get your annotations

In [ ]:
# 1. only path to RData file
path_to_rdata = "data/GSE_bpd/GSE53987_2.rds"
format = False # change this parameter according to the description provided above
adata = rds_to_ann(path_to_rdsmatrix=path_to_rdata, gene_columns=format)

# 2. path to RData files for matrix, gene and sample annotations
path_to_rdata = "data/GSE_bpd/GSE53987_2.rds"
path_to_genes = "data/GSE_bpd/GSE53987_2_annots.rds"
path_to_samples = "data/GSE_bpd/GSE53987_2_metadata.rds"
adata = rds_to_ann(path_to_rdata, path_to_rdsannot=path_to_genes, path_to_rdsmeta=path_to_samples, gene_columns=format)

# 3. path to RData file containing matrix and self extracted annotations
path_to_rdata = "data/GSE_bpd/GSE53987_2.rds"
gene_annot = [] # list of gene annotations, need to have length compatible with provided matrix, or default annotation is used
sample_annot = [] # list of sample annotations, need to have length compatible with provided matrix, or default annotation is used
adata = rds_to_ann(path_to_rdata, gene_annot=gene_annot, sample_annot=sample_annot, gene_columns=format)

### RData annotation extraction Tutorial with rds2py

Tutorial on how to extract annotations from RData files with rds2py. This is the way how it is done in the rsd_to_ann function. Thus, if your data have the same format, the function should work fine with just providing the paths to the RData files. If your data is structured differently, you just have to change the "index" variable in the following code and can use self extracted annotations.

In [ ]:
import rds2py

path_to_genes = "data/GSE_bpd/GSE53987_2_annots.rds"
path_to_samples = "data/GSE_bpd/GSE53987_2_metadata.rds"

r_obj_gene = rds2py.read_rds(path_to_genes)
# The list printed in the following line shows the names of the dictionaries saved in r_obj["data"]
print(r_obj_gene["attributes"]["names"]["data"])
# This should be the index (starting from 0) to the name that stores the information you want to use as annotation
index = 1 # we want to keep "Gene symbol" as gene annotations
gene_annot = r_obj_gene["data"][index]["data"]

r_obj_sample = rds2py.read_rds(path_to_samples)
# The list printed in the following line shows the names of the dictionaries saved in r_obj["data"]
print(r_obj_sample["attributes"]["names"]["data"])
index = 0 # we want to keep "GROUP" as sample annotations
# This should be the index (starting from 0) to the name that stores the information you want to use as annotation
sample_annot = r_obj_sample["data"][index]["data"]

adata = rds_to_ann(path_to_rdata, gene_annot=gene_annot, sample_annot=sample_annot, gene_columns=format)

## Define the remaining parameters for the NMF run

In [ ]:
test_ranks = [3, 4]
tn_initializations = 10
titerations = 100
tseed = 123
tstop_threshold = 40
tnthreads = 1

## Perform the NMF run

In [ ]:
nmf_multiple = nmf.multiple_rank_NMF(adata, # or test_mat or df
                                     test_ranks, 
                                     tn_initializations, 
                                     titerations, 
                                     tseed, 
                                     tstop_threshold, 
                                     tnthreads)

## Working with the NMF results

There are several functions in the multiplerun_NMF_class to work with the results of the NMF run, in the following some useful examples are presented.

In [ ]:
# First, metrics for the different runs are calculated
nmf_multiple.compute_OptKStats_NMF()
# Next, the metrics are compared in a way to find the best suited factorisation rank under the computed ones
nmf_multiple.compute_OptK()

In [ ]:
# Check which feature contributes to which signature
nmf_multiple.WcomputeFeatureStats()
# Create a heatmap showing the jaccard distance between the sets of contributing genes of all signatures
your_path = "SignatureComparison.png"
nmf_multiple.signature_heatmap(your_path)